In [27]:
import polars as pl
import numpy as np
from sklearn.metrics.pairwise import haversine_distances
import os
import pandas as pd
import matplotlib.pyplot as plt
import haversine as hs
import gc
os.getcwd()

'c:\\Users\\digom\\Documents\\GitHub\\fh-energia'

In [28]:
df_cnpj = pl.scan_csv('./data/estabelecimentos/estabelecimentos.csv', has_header=False, infer_schema_length=0)


In [3]:
situ_dict = {'01': 'Nula', '02': 'Ativa', '03': 'Suspensa', '04': 'Inapta', '08': 'Baixada'}

df_cnpj = df_cnpj.select('column_1', 'column_2', 'column_3', 'column_4', 'column_5', 'column_6',
                         'column_11',  'column_12', 'column_13',
                          'column_14', 'column_15', 'column_16', 'column_19', 'column_20', 'column_21')
df_cnpj = df_cnpj.with_columns(pl.concat_str(
        [
            pl.col("column_1"),
            pl.col("column_2"),
            pl.col("column_3"),
        ],
        separator="",
    ).alias('cnpj'),
    pl.concat_str(
        [
            pl.col("column_14"),
            pl.col("column_15"),
            pl.col("column_16"),
        ],
        separator=" ",
    ).alias('endereco'),
    pl.col('column_6').replace(situ_dict).alias('situacao_cadastral'))\
            .rename({'column_4': 'matriz_filial',  'column_19': 'CEP', 'column_12':'CNAE', 'column_13':'CNAE_2', 'column_16': 'numero', 'column_5': 'nome_fantasia',
                     'column_20': 'UF', 'column_21': 'municipio', 'column_11':'CNPJ_DAT_CON'})\
            .with_columns(cnae_2_split=pl.col("CNAE_2").str.split(","))\
            .with_columns(pl.col('CNPJ_DAT_CON').str.to_date('%Y%m%d').alias('CNPJ_DAT_CON'))\
            .with_columns(
                          pl.col('CNPJ_DAT_CON').dt.month().alias('CNPJ_DAT_CON_M'),
                          pl.col('CNPJ_DAT_CON').dt.year().alias('CNPJ_DAT_CON_Y'))\
            .explode('cnae_2_split')\
            .select(['cnpj', 'endereco', 'matriz_filial', 'CEP','CNAE', 'cnae_2_split', 'numero', 'situacao_cadastral', 
                     'nome_fantasia', 'UF', 'municipio', 'CNPJ_DAT_CON', 'CNPJ_DAT_CON_M', 'CNPJ_DAT_CON_Y'])\
            .filter(pl.col('situacao_cadastral').is_in(['Ativa', 'Suspensa', 'Inapta']))

In [4]:
#Filtrando pelo CEP de Sao Paulo
df_cnpj = df_cnpj.filter(pl.col("CEP").str.starts_with("1"))

In [5]:
df_coord_sp = pl.scan_csv('./data/ibge/coord/35.csv', separator=";", infer_schema_length=0)
df_coord_sp = df_coord_sp.filter(~pl.col('COD_ESPECIE').is_in(["1", "2"]))

In [6]:
df_bdgd = pl.scan_csv('data/bdgd/ucmt_pj.csv', separator=";", encoding='utf8-lossy', infer_schema_length=0)
df_bdgd = df_bdgd.with_columns(pl.col('CEP').str.replace('\.', '').str.replace('-', '').alias('CEP'),
                                pl.col('CNAE').str.replace('-', '').str.replace('/', '').alias('CNAE'),
                                pl.col('DAT_CON').str.to_date('%d/%m/%Y').alias('DAT_CON'),)\
                 .with_columns(
                             pl.col('DAT_CON').dt.month().alias('DAT_CON_M'),
                             pl.col('DAT_CON').dt.year().alias('DAT_CON_Y'))\
                 .filter((pl.col('LIV') == '0') & (pl.col("CEP").str.starts_with("1")))\
                 .with_row_index()


In [11]:
df_bdgd_aux = df_bdgd.select('POINT_Y', 'POINT_X').collect().to_numpy()
df_ibge = df_coord_sp.select('LATITUDE', 'LONGITUDE').collect().to_numpy()

In [ ]:
#A function that gets the lowest column value for each row and returns the column index of the lowest value for each row
def nearest_haversine(matrix_1, matrix_2):
    distances = haversine_distances(matrix_1, matrix_2)
    return np.argmin(distances, axis=1)

#split the polars lazyframe df_bdgd into splits of 1000 rows (with the latitudes and longitudes)


df_bdgd_splits = np.array_split(df_bdgd_aux, 100)
nearest_indexes = []
for it_num, arr_split in enumerate(df_bdgd_splits):
    idx = nearest_haversine(arr_split, df_ibge)
    nearest_indexes = [*nearest_indexes, *idx]
    gc.collect()
    print(f"Finalizado it_num: {it_num}")

Finalizado it_num: 0
Finalizado it_num: 1
Finalizado it_num: 2
Finalizado it_num: 3
Finalizado it_num: 4
Finalizado it_num: 5
Finalizado it_num: 6
Finalizado it_num: 7
Finalizado it_num: 8
Finalizado it_num: 9
Finalizado it_num: 10
Finalizado it_num: 11
Finalizado it_num: 12
Finalizado it_num: 13
Finalizado it_num: 14
Finalizado it_num: 15
Finalizado it_num: 16
Finalizado it_num: 17
Finalizado it_num: 18
Finalizado it_num: 19
Finalizado it_num: 20
Finalizado it_num: 21
Finalizado it_num: 22
Finalizado it_num: 23
Finalizado it_num: 24
Finalizado it_num: 25
Finalizado it_num: 26
Finalizado it_num: 27
Finalizado it_num: 28
Finalizado it_num: 29
Finalizado it_num: 30
Finalizado it_num: 31
Finalizado it_num: 32
Finalizado it_num: 33
Finalizado it_num: 34
Finalizado it_num: 35
Finalizado it_num: 36
Finalizado it_num: 37
Finalizado it_num: 38
Finalizado it_num: 39
Finalizado it_num: 40
Finalizado it_num: 41
Finalizado it_num: 42
Finalizado it_num: 43
Finalizado it_num: 44
Finalizado it_num: 4

In [ ]:
#saving pickle of nearest_indexes
np.save('nearest_indexes.npy', nearest_indexes)

: 

In [7]:
#reading nearest_indexes
nearest_indexes = np.load('nearest_indexes.npy')

In [12]:
nearest_indexes
lat_col = []
long_col = []

for idx in nearest_indexes:
    lat_col.append(df_ibge[idx][0])
    long_col.append(df_ibge[idx][1])

lat_long_list = list(zip(lat_col, long_col))

distance_vals = []
for idx, data_bdgd in enumerate(df_bdgd_aux):
    lat_bdgd = float(data_bdgd[0])
    long_bdgd = float(data_bdgd[1])
    lat_ibge = float(lat_long_list[idx][0])
    long_ibge = float(lat_long_list[idx][1])
    distance_vals.append(hs.haversine((lat_bdgd, long_bdgd), (lat_ibge, long_ibge)))

df_bdgd = df_bdgd.with_columns(pl.Series("LATITUDE", lat_col), pl.Series("LONGITUDE", long_col), pl.Series("DISTANCE_IBGE", distance_vals), pl.Series("NEAREST_IBGE", nearest_indexes))

In [13]:
df_bdgd_pandas = df_bdgd.collect().to_pandas()

In [17]:
df_bdgd_pandas

,index,COD_ID,DIST,PN_CON,PAC,CTMT,UNI_TR_AT,SUB,CONJ,MUN,...,DESCR,DATA_BASE,POINT_X,POINT_Y,DAT_CON_M,DAT_CON_Y,LATITUDE,LONGITUDE,DISTANCE_IBGE,NEAREST_IBGE
0,0,41799127,385,2109274265,LEM142109274265,LEM01_AL014,LEM01_ETR01,LEM01,13529,3526704,...,None,31DEC2022:00:00:00.0000000,-47.37992327,-22.17505612,4,2018,-22.175316,-47.380281,0.046818,1346347
1,1,34432850,385,53136023,MOM0653136023,MOM01_AL006,MOM01_ETR01,MOM01,13540,3530805,...,None,31DEC2022:00:00:00.0000000,-46.98270237,-22.43867159,2,2010,-22.439341,-46.982607,0.075078,2864009
2,2,18447660,385,2109250924,SRP062109250924,SRP01_AL006,SRP01_ETR01,SRP01,13585,3547502,...,None,31DEC2022:00:00:00.0000000,-47.50791476,-21.71267878,2,2000,-21.713623,-47.507404,0.117505,1037075
3,3,28780523,385,24038265,JAC1224038265,CAJ01_AL021,CAJ01_ETR03,CAJ01,13470,3524600,...,None,31DEC2022:00:00:00.0000000,-47.99687411,-24.69152068,8,2004,-24.691404,-47.996917,0.013679,1553645
4,4,18001203,385,23642843,PIS0823642843,PIS01_DJ002,PIS01_ETR01,PIS01,13566,3537909,...,None,31DEC2022:00:00:00.0000000,-47.70691033,-23.8541634,7,1998,-23.852495,-47.697627,0.962144,2113059
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41427,41427,347507,5378,986816,9868161,103_BICU_2,13_BICU_1,BICU,12586,3521804,...,None,31DEC2022:00:00:00.0000000,-49.1029188,-23.58234747,1,1964,-23.541379,-45.958995,320.469232,1381913
41428,41428,4002365302,63,3317438,20016311,AND21,AND-ETR102,AND,16440,3509502,...,None,31DEC2022:00:00:00.0000000,-47.0499816,-22.93695314,1,2019,-22.937296,-47.049793,0.042737,2366367
41429,41429,0033890935,63,2262447,13479526,ESP05,ESP-ETR101,ESP,13941,3553708,...,None,31DEC2022:00:00:00.0000000,-48.57595406,-21.359408,2,2009,-21.359552,-48.575376,0.061967,1323991
41430,41430,0008857059,63,738675576,20137036,TAN06,TAN-ETR202,TAN,16441,3509502,...,None,31DEC2022:00:00:00.0000000,-47.02298257,-22.77098744,3,2018,-22.771812,-47.022933,0.091828,2401266


In [16]:
df_bdgd_pandas.loc[df_bdgd_pandas['DISTANCE_IBGE'] < 0.05]

,index,COD_ID,DIST,PN_CON,PAC,CTMT,UNI_TR_AT,SUB,CONJ,MUN,...,DESCR,DATA_BASE,POINT_X,POINT_Y,DAT_CON_M,DAT_CON_Y,LATITUDE,LONGITUDE,DISTANCE_IBGE,NEAREST_IBGE
0,0,41799127,385,2109274265,LEM142109274265,LEM01_AL014,LEM01_ETR01,LEM01,13529,3526704,...,None,31DEC2022:00:00:00.0000000,-47.37992327,-22.17505612,4,2018,-22.175316,-47.380281,0.046818,1346347
3,3,28780523,385,24038265,JAC1224038265,CAJ01_AL021,CAJ01_ETR03,CAJ01,13470,3524600,...,None,31DEC2022:00:00:00.0000000,-47.99687411,-24.69152068,8,2004,-24.691404,-47.996917,0.013679,1553645
5,5,4210042,385,23706455,ARU1123706455,ARR01_AL011,ARR01_ETR02,ARR01,13602,3503307,...,None,31DEC2022:00:00:00.0000000,-47.3934778,-22.36475087,2,1997,-22.364734,-47.393466,0.002234,1619401
8,8,17111404,385,44230100,PER1044230100,PER10,None,PER01,16610,3537602,...,None,31DEC2022:00:00:00.0000000,-47.00296673,-24.31974006,12,1981,-24.319822,-47.003409,0.045731,1440791
9,9,33154058,385,48542753,ARN0548542753,ARN01_AL005,ARN01_ETR01,ARN01,13457,3503802,...,None,31DEC2022:00:00:00.0000000,-47.16211531,-22.55672742,9,2008,-22.556867,-47.162068,0.016263,1208715
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41424,41424,4657236,5216,5133857,41343330757959739MT,652878,451221,35,12761,3541505,...,None,31DEC2022:00:00:00.0000000,-51.83794765,-21.88566309,7,2008,-21.885625,-51.837980,0.005393,2434499
41425,41425,0000915661,63,2419931,13750660,PRG27,PRG-ETR102,PRG,16450,3529302,...,None,31DEC2022:00:00:00.0000000,-48.37068016,-21.60967039,2,2009,-21.609565,-48.370938,0.029118,1216215
41426,41426,0013731181,63,3397142,17412769,PRA13,PRA-ETR101,PRA,13789,3522604,...,None,31DEC2022:00:00:00.0000000,-46.82058841,-22.43616642,5,2011,-22.436009,-46.820469,0.021378,2533684
41428,41428,4002365302,63,3317438,20016311,AND21,AND-ETR102,AND,16440,3509502,...,None,31DEC2022:00:00:00.0000000,-47.0499816,-22.93695314,1,2019,-22.937296,-47.049793,0.042737,2366367


In [21]:
df_bdgd_lazy = pl.LazyFrame(df_bdgd_pandas.loc[df_bdgd_pandas['DISTANCE_IBGE'] < 0.1])

df_cnae_1 = df_cnpj.join(df_bdgd_lazy, on=['CEP', 'CNAE'], how='inner')
df_cnae_2 = df_cnpj.join(df_bdgd_lazy, left_on=['CEP', 'cnae_2_split'], right_on=['CEP', 'CNAE'], how='inner')
#concatenating the two dataframes
df_final = pl.concat([df_cnae_1, df_cnae_2])
df_filter = df_final.filter(pl.col('LGRD').str.contains(pl.col('numero'), literal=True))\
                    .unique(subset=['COD_ID', 'cnpj'])
df_cep_cnae_num = df_filter#.collect()#.to_pandas()
#getting top 1000 rows from df_cep_cnae_num
df_cep_cnae_num_top = df_cep_cnae_num.head(1000)

In [19]:
df_cep_cnae_num.select(pl.len()).collect()

len
u32
2804


In [22]:
df_cep_cnae_num_pandas = df_cep_cnae_num.collect().to_pandas()

In [23]:
df_cep_cnae_num_pandas

,cnpj,endereco,matriz_filial,CEP,CNAE,cnae_2_split,numero,situacao_cadastral,nome_fantasia,UF,...,DESCR,DATA_BASE,POINT_X,POINT_Y,DAT_CON_M,DAT_CON_Y,LATITUDE,LONGITUDE,DISTANCE_IBGE,NEAREST_IBGE
0,08560010000193,ESTRADA DO BAIRRO DA POMPEIA S/N,1,13960000,0151201,0134200,S/N,Ativa,None,SP,...,None,31DEC2022:00:00:00.0000000,-46.48513114,-22.6152299,6,2021,-22.615850,-46.485762,0.094591,1987300
1,43644201000134,RUA FIORAVANTE GIOVANELLI 55,1,13720000,2222600,2229399,55,Ativa,None,SP,...,None,31DEC2022:00:00:00.0000000,-46.92647084,-21.60603855,5,2022,-21.606061,-46.926557,0.009251,3081222
2,08063191000495,SÍTIO SAO SEBASTIAO - ESTRADA DE MIGUELOPOLIS ...,2,14530000,0115600,0111302,S/N,Ativa,None,SP,...,None,31DEC2022:00:00:00.0000000,-48.16942232,-20.12678484,8,2014,-20.126718,-48.169328,0.012337,2627195
3,54185624000150,AVENIDA GRANDES LAGOS 189,1,15775000,1411801,1359600,189,Ativa,None,SP,...,None,31DEC2022:00:00:00.0000000,-50.92208806,-20.19106078,2,1990,-20.190947,-50.922723,0.067461,1056614
4,20051414000157,RUA EUGENIO BERTINI 640,1,13477570,1351100,1412601,640,Ativa,None,SP,...,None,31DEC2022:00:00:00.0000000,-47.29666003,-22.74058108,7,2021,-22.740636,-47.296284,0.039043,2877855
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76888,07948124007660,SITIO SANTA HELENA S/N,2,14530000,0113000,0115600,S/N,Ativa,None,SP,...,None,31DEC2022:00:00:00.0000000,-48.20489212,-20.07785984,10,2021,-20.078139,-48.204517,0.049984,2627209
76889,44589295000159,RODOVIA JOSE FREGONESI S/N,1,14110000,6810201,4110700,S/N,Ativa,None,SP,...,None,31DEC2022:00:00:00.0000000,-47.81104319,-21.22833996,8,2021,-21.228405,-47.811943,0.093545,2169164
76890,09282921000169,SÍTIO HIMALAIA S/N,1,14470000,0134200,None,S/N,Ativa,None,SP,...,None,31DEC2022:00:00:00.0000000,-47.56835004,-20.1515488,12,2016,-20.151301,-47.568436,0.028978,221323
76891,10399441000242,TRAVESSA SEIS ESTRADA VELHA SP PARANA CX DE LU...,2,18195000,0119906,0111302,S/N,Ativa,None,SP,...,None,31DEC2022:00:00:00.0000000,-47.73762942,-23.51848651,1,2012,-23.518342,-47.737388,0.029395,2343856


In [24]:
df_cep_cnae_num_pandas['cnpj'].value_counts().loc[lambda x: x==1].index

Index(['08275863000183', '18134937000115', '23394144000175', '39678153000128',
       '26304765000702', '33893345000170', '12937617000117', '45390232000131',
       '08167846000122', '08590480000280',
       ...
       '09190881000125', '04086214000183', '08477784000406', '02463415000127',
       '52386091000149', '43084466000125', '41810878000170', '53209739000174',
       '08378495000107', '10399441000242'],
      dtype='object', length=22420)

In [25]:
single_cnpj = df_cep_cnae_num_pandas['cnpj'].value_counts().loc[lambda x: x==1].index
df_merge_single_cnpj = df_cep_cnae_num_pandas.loc[df_cep_cnae_num_pandas['cnpj'].isin(single_cnpj)]
df_merge_single_cnpj

,cnpj,endereco,matriz_filial,CEP,CNAE,cnae_2_split,numero,situacao_cadastral,nome_fantasia,UF,...,DESCR,DATA_BASE,POINT_X,POINT_Y,DAT_CON_M,DAT_CON_Y,LATITUDE,LONGITUDE,DISTANCE_IBGE,NEAREST_IBGE
3,54185624000150,AVENIDA GRANDES LAGOS 189,1,15775000,1411801,1359600,189,Ativa,None,SP,...,None,31DEC2022:00:00:00.0000000,-50.92208806,-20.19106078,2,1990,-20.190947,-50.922723,0.067461,1056614
4,20051414000157,RUA EUGENIO BERTINI 640,1,13477570,1351100,1412601,640,Ativa,None,SP,...,None,31DEC2022:00:00:00.0000000,-47.29666003,-22.74058108,7,2021,-22.740636,-47.296284,0.039043,2877855
6,40432544060980,RUA ANTONIO MALAQUIAS PEDROSO 1433,2,14160800,6120501,6110899,1433,Ativa,CLARO S/A,SP,...,None,31DEC2022:00:00:00.0000000,-47.9832643,-21.14150482,8,2020,-21.141466,-47.983336,0.008598,1945890
9,08823200000156,SITIO SAO JOAO S/N,1,13360000,0113000,None,S/N,Ativa,None,SP,...,None,31DEC2022:00:00:00.0000000,-47.53068983,-22.95977634,8,2021,-22.960164,-47.530225,0.064212,1045900
16,11856437000148,RUA DESEMBARGADOR MANOEL CARLOS DE FIGUEIREDO ...,1,15155000,3101200,None,536,Ativa,None,SP,...,None,31DEC2022:00:00:00.0000000,-49.57520651,-20.88590967,7,2010,-20.886020,-49.575040,0.021207,3149423
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76883,20201726009741,SITIO SANTA MARIA S/N,2,14765000,0113000,0116401,S/N,Ativa,None,SP,...,None,31DEC2022:00:00:00.0000000,-48.40589809,-21.02092453,7,2022,-21.020781,-48.405755,0.021801,2439534
76885,08559787000138,SITIO ESTIVA S/N,1,13780000,0119903,0151201,S/N,Ativa,None,SP,...,None,31DEC2022:00:00:00.0000000,-46.73383577,-21.6556575,12,2017,-21.655995,-46.734114,0.047278,2830050
76887,43728245001033,AVENIDA SAO PAULO 1182,2,18013003,7020400,9499500,1182,Ativa,ESCRITORIO REGIONAL SOROCABA,SP,...,None,31DEC2022:00:00:00.0000000,-47.44080703,-23.49771445,4,2021,-23.497762,-47.441553,0.076253,320227
76889,44589295000159,RODOVIA JOSE FREGONESI S/N,1,14110000,6810201,4110700,S/N,Ativa,None,SP,...,None,31DEC2022:00:00:00.0000000,-47.81104319,-21.22833996,8,2021,-21.228405,-47.811943,0.093545,2169164


In [26]:
df_merge_single_cnpj

,cnpj,endereco,matriz_filial,CEP,CNAE,cnae_2_split,numero,situacao_cadastral,nome_fantasia,UF,...,DESCR,DATA_BASE,POINT_X,POINT_Y,DAT_CON_M,DAT_CON_Y,LATITUDE,LONGITUDE,DISTANCE_IBGE,NEAREST_IBGE
3,54185624000150,AVENIDA GRANDES LAGOS 189,1,15775000,1411801,1359600,189,Ativa,None,SP,...,None,31DEC2022:00:00:00.0000000,-50.92208806,-20.19106078,2,1990,-20.190947,-50.922723,0.067461,1056614
4,20051414000157,RUA EUGENIO BERTINI 640,1,13477570,1351100,1412601,640,Ativa,None,SP,...,None,31DEC2022:00:00:00.0000000,-47.29666003,-22.74058108,7,2021,-22.740636,-47.296284,0.039043,2877855
6,40432544060980,RUA ANTONIO MALAQUIAS PEDROSO 1433,2,14160800,6120501,6110899,1433,Ativa,CLARO S/A,SP,...,None,31DEC2022:00:00:00.0000000,-47.9832643,-21.14150482,8,2020,-21.141466,-47.983336,0.008598,1945890
9,08823200000156,SITIO SAO JOAO S/N,1,13360000,0113000,None,S/N,Ativa,None,SP,...,None,31DEC2022:00:00:00.0000000,-47.53068983,-22.95977634,8,2021,-22.960164,-47.530225,0.064212,1045900
16,11856437000148,RUA DESEMBARGADOR MANOEL CARLOS DE FIGUEIREDO ...,1,15155000,3101200,None,536,Ativa,None,SP,...,None,31DEC2022:00:00:00.0000000,-49.57520651,-20.88590967,7,2010,-20.886020,-49.575040,0.021207,3149423
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76883,20201726009741,SITIO SANTA MARIA S/N,2,14765000,0113000,0116401,S/N,Ativa,None,SP,...,None,31DEC2022:00:00:00.0000000,-48.40589809,-21.02092453,7,2022,-21.020781,-48.405755,0.021801,2439534
76885,08559787000138,SITIO ESTIVA S/N,1,13780000,0119903,0151201,S/N,Ativa,None,SP,...,None,31DEC2022:00:00:00.0000000,-46.73383577,-21.6556575,12,2017,-21.655995,-46.734114,0.047278,2830050
76887,43728245001033,AVENIDA SAO PAULO 1182,2,18013003,7020400,9499500,1182,Ativa,ESCRITORIO REGIONAL SOROCABA,SP,...,None,31DEC2022:00:00:00.0000000,-47.44080703,-23.49771445,4,2021,-23.497762,-47.441553,0.076253,320227
76889,44589295000159,RODOVIA JOSE FREGONESI S/N,1,14110000,6810201,4110700,S/N,Ativa,None,SP,...,None,31DEC2022:00:00:00.0000000,-47.81104319,-21.22833996,8,2021,-21.228405,-47.811943,0.093545,2169164
